
# 1) Configuration de PRAW (Python Reddit Api Wrapper)
## !! A faire avant de pouvoir utiliser n'importe laquelle des parties suivantes !!

In [1]:
import praw               # API pour reddit
import pandas as pd       # pour afficher les dictionnaires sous forme de tableaux
import datetime as dt     # Pour convertir la date au bon format
import os
import sys
import shutil

In [2]:
# Créer la variable de configuration pour accéder aux requêtes vers reddit
reddit = praw.Reddit(client_id='BEjar6X3GYV5Vw', \
                     client_secret='lg6D6DXG14FH4kHtGVt7cins5OY', \
                     user_agent='feelcrypt', \
                     username='feelcrypt', \
                     password='spiderminute38')

# 2) Générer la liste des posts à traiter
## !! Ne lancer qu'une fois par subreddit !!

Cette partie va récupérer les 500 top posts du subreddit défini et sauvegarder uniquement les id de ces posts dans une liste de post à traiter (un fichier txt)

## 2.1) Variables globales

In [3]:
# Variables de configuration globale
nb_top_posts = 500  # nombre de posts selectionnés parmi les premiers (limite = 500)
subreddit_title = 'btc' # Définir le titre du subreddit que l'on va cibler

## 2.2) Fonctions

In [4]:
def get_script_dir_path():
    script_dir_path = %pwd 
    script_dir_path += '\\'
    return script_dir_path

In [5]:
# Vérifier si le dossier passé en paramètre (+chemin relatif au dossier du subreddit) existe, sinon le créer
def create_folder(subreddit_path, folder_name):
    folder_name = folder_name
    folder_path = subreddit_path + folder_name + '\\'
    try:
        os.makedirs(folder_path)
        print('Dossier créé : ' + folder_name) # Log
    except FileExistsError:
        print('Dossier déjà existant : ' + folder_name) # Log
        pass
    return folder_path

In [6]:
# Créer la liste des id des posts à traiter
def create_id_todo(subreddit_path):
    # Créer la variable sur le subreddit
    subreddit = reddit.subreddit(subreddit_title)
    
    # Récupérer les n premiers posts de notre subreddit de notre liste
    top_posts = []

    for post in subreddit.top(limit = nb_top_posts):
        top_posts.append(post)

    # Enregistrer les id dans une liste
    top_posts_id = []
    for post in top_posts:
        top_posts_id.append(post.id)

    # Enregistrer la liste des id dans un fichier txt

    # Créer le chemin de la liste d'id à traiter
    file_name = 'id_todo.txt'
    id_todo__path = subreddit_path + file_name

    # Remplir la liste avec les id
    top_posts_id_file = open(id_todo__path,'w')
    for post_id in top_posts_id:
         top_posts_id_file.write(post_id)
         top_posts_id_file.write('\n')
    top_posts_id_file.close()
    
    # Log
    print('Fichier initialisé : ' + file_name)
    
    return id_todo__path

In [7]:
# Créer le fichier pour compter le nombre de commentaires total
# Le créer et l'initialiser à 0 (ecrase et remplace par 0 si le fichier existe déjà)
def create_counter(subreddit_path):
    file_name = 'comments_counter.txt'
    counter__path = subreddit_path + file_name
    nb_file = open(counter__path,'w').close()
    nb_file = open(counter__path,'w')
    nb_file.write(str(0))
    nb_file.close()
    
    # log
    print('Fichier initialisé : ' + file_name)
    
    return counter__path

## 2.3) Script

In [8]:
# Récupérer le chemin du dossier contenant le script python
script_folder = get_script_dir_path()

# Créer un dossier pour le subreddit
subreddit_path = create_folder(script_folder, subreddit_title)

# Vérifier si le dossier comments (chemin relatif) existe, sinon le créer
comments_path = create_folder(subreddit_path, 'comments')

# Vérifier si le dossier comments (chemin relatif) existe, sinon le créer
comments_manager_path = create_folder(subreddit_path, 'comments_manager')

Dossier déjà existant : btc
Dossier déjà existant : comments
Dossier déjà existant : comments_manager


In [9]:
# Créer la liste des id des posts à traiter
id_todo_path = create_id_todo(subreddit_path)

Fichier initialisé : id_todo.txt


In [10]:
# Créer le fichier pour compter le nombre de commentaires total
file_name = 'comments_counter.txt'
counter_path = subreddit_path + file_name
    
if not os.path.isfile(counter_path):
    counter_path = create_counter(subreddit_path)

# 3) Traiter les posts

## 3.1) Variables globales

In [11]:
# Variables globales

scrap_all_comments = True
# Si false : prend seulement les 32 premiers commentaires de chaque post
# Si true : prend tous les commentaires du post

scrap_responses_to_comments = True
# Si false : ne prend que les réponses directes au post
# Si true : prend également en compte les réponses aux commentaires

## 3.2) Functions

In [12]:
# Fonction de conversion pour la date avec timestamp
def get_date(created):
    return dt.date.fromtimestamp(created)

In [13]:
# Ajouter le nb de commentaires traités dans le compteur
def updating_counter(counter_path, nb_comments, current_post_id):
        nb_file = open(counter_path,'r')
        total_comments = int(nb_file.read())
        print("previously total number of comments : " + str (total_comments))
        total_comments += nb_comments
        nb_file.close()
        nb_file = open(counter_path,'w').close()
        nb_file = open(counter_path,'w')
        nb_file.write(str(total_comments))
        nb_file.close()
        
        print(str(nb_comments) + " new comments scrapped from post " + str(current_post_id))
        print('total comments saved = ' + str(total_comments))

In [14]:
# Compter le nombre de lignes dans un fichier
def file_len(fname):
    i = 0
    with open(fname) as f:
        for l in enumerate(f):
            i += 1
    return i

In [15]:
# Compter le nombre de commentaires déjà scrapés pour un post
#def count_already_scrapped_comments(post_id)
    

In [16]:
# Récupérer la liste des commentaires déjà traités depuis le comment_manager
# Et comparer avec la liste récupérée

In [17]:
# Traiter chaque id du fichier todo
def get_comments(id_todo):
    while id_todo:  # Vérifier si la liste n'est pas vide

        # Récupérer le post
        post = reddit.submission(id = id_todo[0])
        current_post_id = post.id
        print('-----------------------------------\n\n')
        print('Starting working on post : ' + str(current_post_id))
        
        # ouvrir le fichier txt pour les commentaires déjà traités
        print("Step 1 ----------------------- ############")
        manager_file_path = comments_manager_path + str(current_post_id)[0:6] + '.txt'
        comment_manager_file = open(manager_file_path,'a')
        comment_manager_file.close()
        
        # compter le nombre de lignes du fichier = le nombre de commentaire déjà traités sur la totalité du du subreddit
        print("Step 2 ----------------------- ############")
        nb_comments_done = file_len(manager_file_path)
        print(manager_file_path)
        
        
        ## Récupérer les commentaires du poste
        
        #Réinitialiser le dictionnaire
        comments_dict = { "created_utc":[], \
                         "body":[], \
                         "score":[], \
                         "nb_replies":[], \
                         "stickied":[], \
                         "author":[], \
                         "id":[], \
                         "post_title":[], \
                         "post_id":[], \
                         "post_link":[]} 
        
        print("Step 3 : count all comments in a post without the replace_more")
        print("nb of comments with variable : " + str(post.num_comments))

        print("Step 4 ----------------------- ############")
        # Se débarasser récursivement de la limite de 32 commentaires par requête
        # NB : Cette partie est la plus longue lorsqu'on scrap un post
        if scrap_all_comments:
            post.comments.replace_more(limit=None)  # prendre en compte les commentaires supp
        else:
            post.comments.replace_more(limit=0)    # ignorer les commentaires supp
            
        print("Step 5 ----------------------- ############")
        # Récupérer la liste des commentaires
        if scrap_responses_to_comments:
            comments_list = post.comments.list()
        else:
            comments_list = post.comments
        
        # Checker s'il y a de nouveaux commentaires sur le post
            # comparer la longueur de la comments_list à la longueur de la liste des comments déjà scrappés
        new_comments_length = len(comments_list)
        print('number of comments on post ' + str(current_post_id)[0:6] +  ' right now : ' + str(new_comments_length))
        
        # Compter le nb de commentaires déjà scrappés
        number_already_scrapped_comments = file_len(manager_file_path)
        print('number of comments already scrapped on this post : ' + str(number_already_scrapped_comments))
        
        bool_new_comments = (new_comments_length > number_already_scrapped_comments)
        print('bool new comments : ' + str(bool_new_comments))
        
        # If there are new comments to scrap
        if bool_new_comments:
            
        # Remplir le dictionnaire de commentaires avec seulement les nouveaux commentaires
        # On check donc si le commentaire n'est pas dans la liste comment_manager_file
            # Réouvrir le fichier des commentaires
            
                
            new_comments_list = []
                      
            for comment in comments_list:
                bool_comment_scrapped = False
                #print("working on comment : " + str(comment.id))
                
                comment_manager_file = open(manager_file_path,'r')
                for comment_id_done in comment_manager_file:
                    #print("id_done = " + str(comment_id_done[0:7]) + " | comment.id = " + str(comment.id))
                    if(str(comment.id) == str(comment_id_done[0:7])):
                        #print("comment already scrapped : " + str(comment_id_done))
                        bool_comment_scrapped = True
                    #else:
                        #print("comment not already scrapped : " + str(comment.id))
                comment_manager_file.close()
                #print(str(comment.id) + " done ? : " + str(bool_comment_scrapped))
                if not bool_comment_scrapped:
                    new_comments_list.append(comment)
                
            comment_manager_file = open(manager_file_path,'a')    
            for comment in new_comments_list:
                #print("--- this is a comment id from the new_comment_list : " + str(comment.id))
                origin_post = comment.submission
                comments_dict["created_utc"].append(comment.created_utc)
                comments_dict["body"].append(comment.body)
                comments_dict["score"].append(comment.score)
                comments_dict["nb_replies"].append(len(comment.replies))
                comments_dict["stickied"].append(comment.stickied)
                comments_dict["author"].append(comment.author)
                comments_dict["id"].append(comment.id)
                comments_dict["post_title"].append(origin_post.title)
                comments_dict["post_id"].append(origin_post.id)
                comments_dict["post_link"].append(origin_post.url)

                # Ajouter l'id du commentaires à la liste des commentaires scrappés
                comment_manager_file.write(comment.id + '\n')

            # mettre la data au format pandas (qui permet de faire un "tableur" à partir du dictionnaire)
            comments_data = pd.DataFrame(comments_dict)

            # Créer la liste des dates converties et la sauvgarder dans la variable _timstamp
            # created_utc est la colonne contenant les dates au mauvais format
            _timestamp = comments_data["created_utc"].apply(get_date)

            # ajouter la liste à une nouvelle colonne appelée timestamp
            comments_data = comments_data.assign(date = _timestamp)

            # Supprimer la colonne du temps inutile
            comments_data = comments_data.drop(columns="created_utc")

            # Déplacer la date en première position
            colonnes = comments_data.columns.tolist()
            colonnes = colonnes[-1:] + colonnes[:-1]
            comments_data = comments_data[colonnes]

            # Trier par la colonne date
            comments_sorted = comments_data.sort_values(by=['date'])

            ## Extraire plusieurs dataframe qui représentent chacun une date avec tous les commentaires dedans
            # Récupérer la liste des dates et indexer par dates
            comments_sorted.set_index(keys=['date'], drop=False,inplace=True)
            dates = comments_sorted['date'].unique().tolist()

            # Enregistrer dans une liste contenant chaque dataframe (1 data frame = 1 date)
            comments_splitperday = []
            for date in dates:
                comments_per_day = pd.DataFrame(comments_sorted.loc[comments_sorted.date == date])
                comments_splitperday.append(comments_per_day)

            # Enregistrer chaque dataframe dans un fichier csv
            for dataframe in comments_splitperday:
                # Récupérer la date du dataframe supprimer la colonne date
                date = str(dataframe.date.iloc[0])
                dataframe = dataframe.drop(columns="date")
                csv_path = comments_path + date + '.csv'

                # Check if file is empty (used to sed header or not)
                csv_exists = os.path.exists(csv_path.replace(os.sep, '/'))

                # Enregistrer au format csv avec pour nom la date
                dataframe.to_csv(csv_path, ';', mode='a', index=False, header= not csv_exists) 
                
                #----- Fin du traitement des commentaires d'un post

            # Retirer le premier id de la liste, le laisser à la fin, si le while est interrompu, il sera retiré alors que le post n'aura pas été traité
            id_todo.pop(0)

            # Ajouter l'id du post lu dans la liste des id_done si le post n'avait pas déjà été scrappé
            if (number_already_scrapped_comments <= 0):
                id_done_path = subreddit_path + "id_done.txt"
                with open(id_done_path, "a") as file:
                    file.write(current_post_id)

            # Enregistrer la nouvelle liste des id_todo dans le fichier txt (ou le créer s'il n'existe pas encore)
            with open(id_todo_path, "r") as file:
                data = file.read()
            with open(id_todo_path, "w") as file:
                for post_id in id_todo:
                    file.write(post_id)

            # Ajouter le nb de commentaires traités dans le compteur
            updating_counter(counter_path, len(comments_data.index), current_post_id)

            # Fermer le fichier comment_manager
            comment_manager_file.close()
            
        else:
            # Retirer le premier id de la liste, le laisser à la fin, si le while est interrompu, il sera retiré alors que le post n'aura pas été traité
            id_todo.pop(0)

            # Enregistrer la nouvelle liste des id_todo dans le fichier txt (ou le créer s'il n'existe pas encore)
            with open(id_todo_path, "r") as file:
                data = file.read()
            with open(id_todo_path, "w") as file:
                for post_id in id_todo:
                    file.write(post_id)
                    
            # Fermer le fichier comment_manager
            comment_manager_file.close()
    else:
        print('============================================')
        print("La liste est vide, tout a été traité")

## 3.3) Script

In [18]:
# Ouvrir la liste de posts pas encore faits
with open(id_todo_path) as file:
  id_todo = file.readlines()

In [ ]:
# Traiter chaque id du fichier todo
get_comments(id_todo)

-----------------------------------


Starting working on post : 7eil12

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\7eil12.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 1162
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7eil12 right now : 1192
number of comments already scrapped on this post : 1192
bool new comments : False
-----------------------------------


Starting working on post : 7opi7w

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\7opi7w.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 747
Step 4 ----------------------- ############
Step 5 ---------------

nb of comments with variable : 267
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7l10a4 right now : 270
number of comments already scrapped on this post : 270
bool new comments : False
-----------------------------------


Starting working on post : 7mkyjy

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\7mkyjy.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 320
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7mkyjy right now : 329
number of comments already scrapped on this post : 329
bool new comments : False
-----------------------------------


Starting working on post : 7lfyrf

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\L

nb of comments with variable : 516
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7ofrqf right now : 526
number of comments already scrapped on this post : 526
bool new comments : False
-----------------------------------


Starting working on post : 7iyyti

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\7iyyti.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 432
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7iyyti right now : 440
number of comments already scrapped on this post : 440
bool new comments : False
-----------------------------------


Starting working on post : 7scgqn

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\L

nb of comments with variable : 521
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7x5opq right now : 536
number of comments already scrapped on this post : 536
bool new comments : False
-----------------------------------


Starting working on post : 7dv508

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\7dv508.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 311
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7dv508 right now : 316
number of comments already scrapped on this post : 316
bool new comments : False
-----------------------------------


Starting working on post : 7e8ke5

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\L

nb of comments with variable : 609
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 6qy4df right now : 593
number of comments already scrapped on this post : 593
bool new comments : False
-----------------------------------


Starting working on post : 6vbrt6

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\6vbrt6.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 236
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 6vbrt6 right now : 237
number of comments already scrapped on this post : 237
bool new comments : False
-----------------------------------


Starting working on post : 7qix0z

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\L

...

nb of comments with variable : 261
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 6wy0ss right now : 275
number of comments already scrapped on this post : 275
bool new comments : False
-----------------------------------


Starting working on post : 8w1rew

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\8w1rew.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 122
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 8w1rew right now : 123
number of comments already scrapped on this post : 123
bool new comments : False
-----------------------------------


Starting working on post : 8ef1hv

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\L

nb of comments with variable : 71
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7l1snq right now : 72
number of comments already scrapped on this post : 72
bool new comments : False
-----------------------------------


Starting working on post : 6ettpe

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\6ettpe.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 144
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 6ettpe right now : 157
number of comments already scrapped on this post : 157
bool new comments : False
-----------------------------------


Starting working on post : 8kem0p

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Loui

nb of comments with variable : 528
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 8f9cs3 right now : 540
number of comments already scrapped on this post : 540
bool new comments : False
-----------------------------------


Starting working on post : 87jpkr

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\87jpkr.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 124
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 87jpkr right now : 124
number of comments already scrapped on this post : 124
bool new comments : False
-----------------------------------


Starting working on post : 6x83ip

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\L

nb of comments with variable : 370
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 5zgefe right now : 403
number of comments already scrapped on this post : 403
bool new comments : False
-----------------------------------


Starting working on post : 85uo6j

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\85uo6j.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 368
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 85uo6j right now : 376
number of comments already scrapped on this post : 376
bool new comments : False
-----------------------------------


Starting working on post : 7m7l72

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\L

nb of comments with variable : 83
Step 4 ----------------------- ############
Step 5 ----------------------- ############
number of comments on post 7k9tag right now : 81
number of comments already scrapped on this post : 81
bool new comments : False
-----------------------------------


Starting working on post : 6jqrub

Step 1 ----------------------- ############
Step 2 ----------------------- ############
C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\6jqrub.txt
Step 3 : count all comments in a post without the replace_more
nb of comments with variable : 544
Step 4 ----------------------- ############


In [ ]:

len1 = file_len(r'C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V4\btc\comments_manager\7hzklb.txt')
print(str(len1))

# 4) Supprimer toute la data
utile uniquement en debbug, permet de supprimer tous les dossiers générés et leur contenu

In [ ]:
import os

In [ ]:
def Delete_all_data():
    if os.path.isdir(subreddit_path):
        shutil.rmtree(subreddit_path)
        print('Dossier supprimé : ' + subreddit_path)
    else:
        print('Dossier inexistant : ' + subreddit_path)

In [ ]:
#Delete_all_data()

In [ ]:
"""
comment_manager_file = open("C:\\Users\\Louis\\feelcrypt\\FeelCrypt\\scrapping_reddit\\scrapping_V4\\btc\\comments_manager\\7eil12.txt",'r')
for comment_id_done in comment_manager_file:
    print(str(comment_id_done[0:6]))
comment_manager_file.close()
"""